#  IA para Redes de Suministro 

👤 **Autor:** John Leonardo Vargas Mesa  
🔗 [LinkedIn](https://www.linkedin.com/in/leonardovargas/) | [GitHub](https://github.com/LeStark)  

## 📂 Repositorio en GitHub  
- 📓 **Notebooks:** [Acceder aquí](https://github.com/LeStark/Cursos/tree/main/02%20-%20IA4SC)  
- 📑 **Data sets:** [Acceder aquí](https://github.com/LeStark/Cursos/tree/main/00%20-%20Data/02%20-%20SC)  
---

# 📘 Notebook 5 – Introducción a Natural Lenguage Processing para redes de suministro

Este notebook introduce los conceptos fundamentales del **Procesamiento de Lenguaje Natural (NLP)** a través de un caso práctico de **análisis de sentimientos** utilizando reseñas de productos de Amazon.

A lo largo del notebook se recorren las principales etapas del flujo de trabajo en NLP, desde la limpieza del texto hasta la interpretación de los resultados del modelo.

## 🧩 **Contenido del Notebook**

1. **Carga y Exploración del Dataset:**  
   Se utiliza un conjunto de reseñas reales de Amazon que incluye texto y calificación (`overall`). Se realiza una exploración inicial para comprender la estructura de los datos.

2. **Conversión de Calificaciones a Sentimientos:**  
   Se crea una columna categórica (`sentiment`) basada en la calificación:  
   - `Positive` (≥ 4)  
   - `Neutral` (= 3)  
   - `Negative` (≤ 2)

3. **Codificación y Análisis de Balance:**  
   Se codifican las etiquetas numéricamente (`LabelEncoder`) y se visualiza la distribución de clases para detectar posibles desbalances en el dataset.

4. **Preprocesamiento del Texto:**  
   Se aplican técnicas de limpieza y normalización como:
   - Conversión a minúsculas  
   - Eliminación de signos, números y URLs  
   - Tokenización  
   - Eliminación de *stopwords*  
   - Lematización con spaCy  

   El resultado se almacena en la columna `clean_text`.

5. **Balanceo del Dataset:**  
   Se realiza un **submuestreo** para igualar la cantidad de reseñas por clase, creando un conjunto balanceado llamado `amazon_reviews_balanced`.

6. **Vectorización (TF-IDF):**  
   Se transforma el texto limpio en vectores numéricos mediante **TF-IDF**, limitando el vocabulario a las palabras más relevantes.

7. **Entrenamiento del Modelo:**  
   Se entrena una **Regresión Logística** con ajuste de pesos (`class_weight='balanced'`) para manejar el desbalance de clases.

8. **Evaluación del Modelo:**  
   Se generan métricas de rendimiento (precision, recall, f1-score) y se visualiza la matriz de confusión para interpretar los resultados.

9. **Predicción sobre Nuevos Textos:**  
   Se define una función `predict_sentiment()` que integra todo el pipeline (preprocesamiento + vectorización + modelo + decodificación) para clasificar nuevas reseñas.

10. **Prueba con Datos Nuevos:**  
    Se aplica la función a un archivo de reseñas de muestra (`sample_amazon_reviews.csv`) y se observan las predicciones de sentimiento.

## 🎯 **Objetivo de Aprendizaje**

Al finalizar el notebook, el estudiante podrá:
- Comprender el flujo completo de un proyecto de NLP.  
- Implementar un pipeline básico de análisis de sentimientos.  
- Aplicar técnicas de preprocesamiento y vectorización de texto.  
- Entrenar, evaluar y reutilizar un modelo de Machine Learning aplicado a lenguaje natural.

## 🧾 **Herramientas Utilizadas**
- **Python:** procesamiento y modelado  
- **NLTK / spaCy:** limpieza, tokenización y lematización  
- **scikit-learn:** vectorización, modelado y evaluación  
- **Matplotlib / Seaborn:** visualización de resultados  

*Este notebook sirve como punto de partida para comprender los fundamentos prácticos del procesamiento de lenguaje natural y la construcción de modelos de análisis de texto.*


In [ ]:
# Instala las bibliotecas necesarias si no las tienes instaladas
!pip install pandas scikit-learn nltk spacy matplotlib seaborn imblearn

In [ ]:
!pip install "numpy<2"

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
# --- Manipulación y análisis de datos ---
import pandas as pd
import numpy as np

# --- Procesamiento de texto ---
import re
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# --- Modelado y vectorización ---
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# --- Manejo de desbalance ---
from imblearn.over_sampling import RandomOverSampler

# --- Visualización ---
import matplotlib.pyplot as plt
import seaborn as sns

# --- Descarga de recursos NLTK ---
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# --- Carga del modelo de lenguaje de spaCy ---
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    from spacy.cli import download
    download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")


**Carga y Exploración Inicial del Dataset**

En esta sección cargamos el dataset **Amazon Reviews**, que contiene opiniones reales de usuarios sobre distintos productos publicados en la plataforma de Amazon.  
El archivo se encuentra alojado en un repositorio público de GitHub y se importa directamente desde su URL en formato CSV.

###  **Descripción del Dataset**

El dataset **Amazon Reviews** contiene reseñas de productos realizadas por usuarios en la plataforma de Amazon.  
Incluye tanto el texto de la reseña como métricas de utilidad y calificaciones otorgadas por otros usuarios.

A continuación se describen las columnas principales:

| **Columna** | **Descripción** |
|--------------|-----------------|
| `reviewerName` | Nombre del usuario que escribió la reseña. |
| `overall` | Calificación general otorgada al producto, en una escala de **1 a 5** (1 = muy mala, 5 = excelente). |
| `reviewText` | Texto libre con la opinión o experiencia del usuario respecto al producto. |
| `reviewTime` | Fecha en la que la reseña fue publicada. |
| `day_diff` | Diferencia en días entre la fecha de la reseña y una fecha de referencia (útil para análisis temporales). |
| `helpful_yes` | Número de votos que consideraron la reseña como útil. |
| `helpful_no` | Número de votos que consideraron la reseña como **no útil**. |
| `total_vote` | Total de votos recibidos (`helpful_yes + helpful_no`). |
| `score_pos_neg_diff` | Diferencia entre votos positivos y negativos (mide la percepción general de utilidad). |
| `score_average_rating` | Promedio de la puntuación basada en votos de utilidad. |
| `wilson_lower_bound` | Estimación estadística de la utilidad de la reseña considerando la incertidumbre de los votos (basada en el método de Wilson). |

Este conjunto de datos es ideal para realizar **análisis de sentimientos** y explorar la relación entre las calificaciones numéricas (`overall`) y las opiniones escritas (`reviewText`), complementadas con métricas de confianza y popularidad de las reseñas.




In [ ]:
#TODO: cargar dataset "https://raw.githubusercontent.com/LeStark/Cursos/refs/heads/main/00%20-%20Data/03%20-%20NLP/amazon_reviews.csv"


### **Selección de Columnas Relevantes**

En esta etapa se seleccionan únicamente las columnas necesarias para el análisis de sentimientos:  
- `reviewText`: texto de la reseña.  
- `overall`: calificación numérica otorgada por el usuario.  

Esto simplifica el dataset para enfocarnos en las variables clave del modelo.


In [ ]:
#TODO: seleccionar solo las columnas de interes

### **Conversión de Calificaciones Numéricas a Categorías de Sentimiento**

En esta sección se transforma la columna `overall` (puntuación de 1 a 5) en una variable categórica llamada `sentiment`.  
Esta nueva columna clasifica cada reseña según el tono general de la opinión del usuario:

- **Positive (≥ 4):** opiniones favorables o muy satisfechas.  
- **Neutral (= 3):** opiniones intermedias o sin una posición clara.  
- **Negative (≤ 2):** opiniones desfavorables o con experiencias negativas.  

Esta conversión facilita el entrenamiento de modelos de **análisis de sentimientos**, al convertir las valoraciones numéricas en etiquetas textuales comprensibles.


In [ ]:
# --- Conversión de 'overall' a etiquetas de sentimiento ---
#TODO: definir una función para convertir 'overall' a 'positive', 'neutral', 'negative'

# Crear la nueva columna
amazon_reviews["sentiment"] = amazon_reviews["overall"].apply(get_sentiment)

# Mostrar una muestra
amazon_reviews.head()


### **Codificación de las Etiquetas de Sentimiento**

En esta etapa se convierte la columna `sentiment`, que contiene valores categóricos (*positive*, *neutral*, *negative*), en valores numéricos mediante la clase `LabelEncoder` de **scikit-learn**.  

Esto es necesario porque los algoritmos de Machine Learning trabajan con variables numéricas.  
Cada etiqueta de texto se transforma en un número entero único, por ejemplo:

- **negative → 0**  
- **neutral → 1**  
- **positive → 2**

El resultado se almacena en una nueva columna llamada `sentiment_encoded`, que servirá como variable objetivo (`y`) durante el entrenamiento del modelo.


In [ ]:

# --- Codificar la columna 'sentiment' ---
le = #usar el codiicador de etiquetas de sklearn

# Ajustar el codificador y transformar la columna

#TODO: definir la nueva columna con los valores codificados llamada sentiment_encoded

# Mostrar cómo se asignaron los valores
for clase, codigo in zip(le.classes_, range(len(le.classes_))):
    print(f"{clase}: {codigo}")

# Vista rápida del DataFrame
amazon_reviews.head()


###  **Distribución de Sentimientos y Análisis de Balance de Clases**

En esta sección se visualiza la cantidad de reseñas por categoría de sentimiento (*positive*, *neutral*, *negative*).  
El gráfico de barras permite identificar si existe **desbalance de clases**, es decir, si alguna categoría tiene una cantidad de muestras mucho mayor que las demás.

Este análisis es fundamental antes de entrenar el modelo, ya que un conjunto de datos desequilibrado puede generar sesgos en las predicciones.  
Por ejemplo, si la mayoría de reseñas son positivas, el modelo tenderá a clasificar casi todo como *positive*, afectando la precisión en las clases minoritarias.

La información obtenida aquí servirá para decidir estrategias de balanceo como:
- **Submuestreo o sobremuestreo** de clases.  
- **Ajuste de pesos de clase** durante el entrenamiento.  
- **Evaluación con métricas balanceadas** (precision, recall, F1-score).


In [ ]:


# Contar cuántas reseñas hay por sentimiento
sentiment_counts = amazon_reviews["sentiment"].value_counts().sort_index()

# Crear gráfico de barras
plt.figure(figsize=(7,5))
sns.barplot(x=sentiment_counts.index, y=sentiment_counts.values, palette=["red", "gray", "green"])

# Etiquetas y título
plt.title("Distribución de Sentimientos en Amazon Reviews", fontsize=14)
plt.xlabel("Sentimiento", fontsize=12)
plt.ylabel("Número de Reseñas", fontsize=12)

# Mostrar los valores encima de cada barra
for i, value in enumerate(sentiment_counts.values):
    plt.text(i, value + (value*0.01), str(value), ha="center", va="bottom", fontsize=10)

plt.show()


### **Preprocesamiento del Texto**

El preprocesamiento es una etapa esencial en cualquier proyecto de **Procesamiento de Lenguaje Natural (NLP)**, ya que prepara los textos para que los algoritmos puedan interpretarlos correctamente.  
En esta celda se aplican diferentes técnicas de limpieza y normalización al texto de las reseñas.

#### **Pasos realizados:**

1. **Descarga de recursos lingüísticos:**  
   Se descargan los diccionarios de *stopwords* (palabras vacías como “the”, “and”, “is”) y el tokenizador de `nltk`.  
   Además, se carga el modelo de lenguaje de **spaCy** (`en_core_web_sm`) para poder realizar la lematización.

2. **Conversión a minúsculas:**  
   Unifica el texto y evita que palabras como *“Good”* y *“good”* se traten como diferentes.

3. **Eliminación de ruido:**  
   Se remueven URLs, signos de puntuación, números y otros caracteres que no aportan significado.

4. **Tokenización:**  
   El texto se divide en palabras individuales (*tokens*) para su análisis.

5. **Eliminación de stopwords:**  
   Se eliminan palabras comunes sin valor semántico relevante para el análisis (por ejemplo: *“this”, “that”, “was”*).

6. **Lematización:**  
   Cada palabra se transforma en su forma base o raíz (*“running” → “run”*), lo que ayuda a reducir la dimensionalidad del texto.

7. **Reconstrucción del texto limpio:**  
   Finalmente, las palabras procesadas se unen nuevamente en una cadena, generando una nueva columna llamada `clean_text`.

Esta nueva versión del texto será la base para la **vectorización** y el **entrenamiento del modelo de análisis de sentimientos**, garantizando que los datos estén homogéneos y sin ruido.


In [ ]:
# --- PREPROCESAMIENTO DE TEXTO PARA NLP ---

# En esta sección se realiza la limpieza y normalización del texto.
# Este paso es fundamental antes de entrenar un modelo de NLP, 
# ya que los algoritmos no pueden trabajar directamente con texto sin estructurar.

# -------------------------------------------------------------
#  1. Descarga de recursos necesarios (solo la primera vez)
# -------------------------------------------------------------
# 'stopwords': lista de palabras vacías como "the", "and", "is"
# 'punkt': modelo de tokenización para dividir el texto en palabras
# nltk.download('stopwords')
# nltk.download('punkt')

# Cargar el modelo de lenguaje de spaCy (usado para la lematización)
nlp = spacy.load("en_core_web_sm")

# -------------------------------------------------------------
# 2. Definición de palabras vacías (stopwords)
# -------------------------------------------------------------
# Estas palabras no aportan información relevante al análisis de sentimiento,
# por lo que se eliminan del texto.
stop_words = set(stopwords.words("english"))

# -------------------------------------------------------------
# 3. Función de preprocesamiento
# -------------------------------------------------------------
def preprocess_text(text):
    """
    Esta función limpia y normaliza un texto en varios pasos:
    1. Convierte el texto a minúsculas.
    2. Elimina URLs, caracteres especiales, números y signos.
    3. Tokeniza el texto (divide en palabras).
    4. Elimina stopwords y palabras muy cortas.
    5. Aplica lematización con spaCy (reduce palabras a su raíz).
    6. Reconstruye el texto limpio.
    """
    
    # 1️ Pasar a minúsculas
    text = text.lower()
    
    # 2️ Eliminar URLs, menciones, signos y números
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # URLs
    text = re.sub(r"[^a-z\s]", "", text)                 # Solo letras
    
    # 3️ Tokenización (dividir en palabras)
    #TODO
    
    # 4️ Eliminar stopwords y palabras de longitud < 3
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    
    # 5️ Lematización con spaCy
    doc = nlp(" ".join(tokens))
    lemmas = [token.lemma_ for token in doc]
    
    # 6️ Reconstruir el texto limpio
    return " ".join(lemmas)

# -------------------------------------------------------------
# 4. Aplicar el preprocesamiento al dataset
# -------------------------------------------------------------
# Convertimos la columna 'reviewText' en texto limpio y normalizado.
# Este proceso puede tardar algunos minutos si el dataset es grande.
#TODO

# -------------------------------------------------------------
# 5. Visualizar los resultados
# -------------------------------------------------------------
# Mostramos una muestra comparando el texto original con el texto procesado.
amazon_reviews[["reviewText", "clean_text"]].head()



### **Balanceo del Dataset mediante Submuestreo**

En esta etapa se busca **equilibrar la cantidad de reseñas por cada categoría de sentimiento** para evitar que el modelo se sesgue hacia la clase mayoritaria (por lo general, *positive*).  

#### **Procedimiento:**
1. Se identifica el número mínimo de muestras entre las clases (en este caso, la cantidad de reseñas *negative*).  
2. Se selecciona al azar la misma cantidad de reseñas *positive* y *neutral* para igualar el tamaño de cada grupo.  
   - En el caso de las reseñas *neutral*, se permite el muestreo con reemplazo si son muy pocas.  
3. Se combinan los tres subconjuntos y se reordenan aleatoriamente para crear un nuevo DataFrame llamado `amazon_reviews_balanced`.  

#### **Resultado:**
El nuevo dataset tiene la misma cantidad de ejemplos por clase (*positive*, *neutral*, *negative*), lo cual facilita que el modelo aprenda de forma equitativa y reduzca el sesgo hacia una categoría dominante.

La gráfica resultante muestra visualmente la distribución balanceada de los sentimientos.


In [ ]:
# Cantidad mínima entre clases (usaremos la de 'negative')
min_count = amazon_reviews["sentiment"].value_counts()["negative"]

# Separar por clase
df_pos = amazon_reviews[amazon_reviews["sentiment"] == "positive"].sample(n=min_count, random_state=42)
df_neu = amazon_reviews[amazon_reviews["sentiment"] == "neutral"].sample(n=min_count, random_state=42, replace=True)  # opcional: con reemplazo si hay pocas
df_neg = amazon_reviews[amazon_reviews["sentiment"] == "negative"]

# Unir los subconjuntos balanceados
amazon_reviews_balanced = pd.concat([df_pos, df_neu, df_neg], axis=0).sample(frac=1, random_state=42).reset_index(drop=True)

# Verificar el nuevo balance
print("🔹 Distribución balanceada:")
print(amazon_reviews_balanced["sentiment"].value_counts())

# (Opcional) Visualizar
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
sns.countplot(x="sentiment", data=amazon_reviews_balanced, palette=["red", "gray", "green"])
plt.title("Dataset Balanceado (Submuestreo)")
plt.show()


### **Vectorización del Texto con TF-IDF**

En esta sección se transforma el texto limpio (`clean_text`) en una representación numérica que los modelos de Machine Learning puedan interpretar.  
Para ello se utiliza la técnica **TF-IDF (Term Frequency – Inverse Document Frequency)**, que mide la importancia de cada palabra en relación con el conjunto de documentos.

#### **Proceso:**
1. Se crea un objeto `TfidfVectorizer` que convierte las palabras en vectores numéricos.  
   - El parámetro `max_features=1000` limita el vocabulario a las 1000 palabras más relevantes, evitando un modelo demasiado grande.  
2. Se aplica el método `fit_transform()` sobre la columna `clean_text` para generar la matriz `X`, donde:
   - Cada fila representa una reseña.  
   - Cada columna representa una palabra del vocabulario.  
   - Los valores reflejan el peso TF-IDF de cada palabra.  
3. Finalmente, la variable `y` almacena las etiquetas numéricas (`sentiment_encoded`) que corresponden al sentimiento de cada reseña.

El resultado es una estructura matricial donde los textos quedan representados de forma cuantitativa, lista para alimentar al modelo de clasificación.


In [ ]:

# Crear el vectorizador
vectorizer = TfidfVectorizer(max_features=1000)  # puedes ajustar el número de características

# Ajustar y transformar el texto limpio
X = vectorizer.fit_transform(amazon_reviews_balanced["clean_text"])

# Variable objetivo
y = amazon_reviews_balanced["sentiment_encoded"]

In [ ]:

# Elegir un índice aleatorio o fijo para mostrar
idx = np.random.randint(0, X.shape[0])

# Texto original y limpio
print("🔹 Review original:")
print(amazon_reviews.loc[idx, "reviewText"])
print("\n🔹 Texto preprocesado:")
print(amazon_reviews.loc[idx, "clean_text"])
print("\n🔹 Sentimiento:", amazon_reviews.loc[idx, "sentiment"])

# Obtener las palabras del vocabulario
feature_names = vectorizer.get_feature_names_out()

# Convertir el vector a un DataFrame legible
vector_df = pd.DataFrame(
    X[idx].toarray().flatten(),
    index=feature_names,
    columns=["TF-IDF Value"]
)

# Mostrar solo las palabras con peso no cero
vector_df = vector_df[vector_df["TF-IDF Value"] > 0].sort_values(by="TF-IDF Value", ascending=False)

print("\n🔹 Palabras más relevantes en este review:")
display(vector_df.head(15))


### **División del Conjunto de Datos**

Se divide el dataset en dos subconjuntos:  
- **80%** para entrenamiento (`X_train`, `y_train`)  
- **20%** para prueba (`X_test`, `y_test`)  

El parámetro `stratify=y` garantiza que la proporción de clases se mantenga equilibrada en ambos conjuntos.


In [ ]:
# Dividir en conjunto de entrenamiento y prueba
#TODO

### **Entrenamiento del Modelo de Clasificación**

Se entrena un modelo de **Regresión Logística** para predecir el sentimiento de las reseñas.  
El parámetro `class_weight='balanced'` ajusta automáticamente el peso de cada clase para compensar posibles desbalances en los datos.


In [ ]:
model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train, y_train)

### **Evaluación del Modelo**

Se generan las métricas de desempeño del modelo mediante `classification_report`, que muestra **precisión**, **recobrado (recall)** y **F1-score** por clase.  
Además, la **matriz de confusión** permite visualizar los aciertos y errores en las predicciones de cada categoría de sentimiento.


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Predicciones
y_pred = model.predict(X_test)

# Reporte de métricas
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred), display_labels=le.classes_)
disp.plot(cmap="Blues")
plt.show()

### **Prueba del Modelo con Nuevas Reseñas**

En esta sección se prueban ejemplos de texto reales para evaluar el comportamiento del modelo entrenado.  
Cada reseña se limpia con la misma función de preprocesamiento y luego se transforma en un vector TF-IDF antes de ser clasificada.

El modelo devuelve la categoría de sentimiento predicha (**positive**, **neutral** o **negative**), lo que permite comprobar de manera práctica cómo interpreta nuevas opiniones de usuarios.


In [ ]:
reseña_1 = "I bought 2 of those SanDisk 32 GB microSD , used them on my Galaxy Note and Galaxy S4First one , my phone started saying it was removed , then recognize it again :) then diedI thought it's just a luck , plugged in the 2nd one :) stayed for about 2 months and died suddenly ! and lost everythingnever buying from SanDisk again .. ever"
reseña_2 = "This product is amazing, I totally recommend it!"
nuevo_texto = [reseña_2]
nuevo_texto_limpio = [preprocess_text(t) for t in nuevo_texto]
nuevo_vector = vectorizer.transform(nuevo_texto_limpio)
pred = model.predict(nuevo_vector)
print("Predicción:", le.inverse_transform(pred)[0])

In [ ]:
import joblib

joblib.dump(model, "sentiment_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
joblib.dump(le, "label_encoder.pkl")
